In [13]:
using Utils, Plots, LinearAlgebra
using DifferentialEquations
include("./Rocket_Acceleration.jl")
include("./Quaternions.jl")

In [ ]:
module scp_new_problem
include("./6dof fixed t_burn udotdot.jl")
end # module

using .scp_new_problem

In [ ]:
solution = scp_new_problem.solve(:ptr); # Remember J is augmented cost function

In [ ]:
scp_new_problem.print(solution)
scp_new_problem.plot(solution)
# scp_new_problem.save(solution)

In [ ]:
t_burn = 3.45

t_plot = LinRange(0, 1, 1000) * t_burn
Plots.plot(t_plot, [rad2deg(norm(sample(solution.xc, k)[17:19])) for k in t_plot / t_burn], title="TVC Angular Velocity (Degrees)", label = "Continuous")

t_plot = solution.td * t_burn
Plots.plot!(t_plot, [rad2deg(norm(sample(solution.xc, k)[17:19])) for k in t_plot / t_burn], label = "Discrete")

In [ ]:
solution.xd[4:6, 1] + Acceleration(dt) * [0; 0; 1] * dt + g * dt

In [ ]:
t_burn = 3.45
g = [0; 0; -9.80655]

function f!(dx, x, p, t)
    t_burn = 3.45

    r = x[1:3]
    v = x[4:6]
    quat = x[7:10]
    w = x[11:13]
    T = x[14:16]
    T_dot = x[17:19]

    Control = sample(solution.uc, t / t_burn) # if false, false

    Id = Diagonal([0.2, 0.2, 0.04])
    invId = Diagonal([5.0, 5.0, 25.0])

    dx[1:3] = v
    dx[4:6] = g + rotate(quat, T) * Acceleration(t)
    dx[7:10] = 1/2 * quatL(quat) * [0; w]
    dx[11:13] = invId * ([0; 0; -0.5] × T * Acceleration(t) + [0; 0; Control[4]] - cross(w, Id * w))
    dx[14:16] = T_dot
    dx[17:19] = Control[1:3]
end

prob = ODEProblem(f!, solution.xd[:, 1], (0.0, t_burn))
sol = DifferentialEquations.solve(prob, reltol=1e-8, abstol=1e-8)

xc = sol(solution.td * t_burn).u
xmatrix = transpose(reduce(hcat, xc))
# plot(xmatrix[:, 1], xmatrix[:, 2], xmatrix[:, 3])
# # plot(sol)

In [ ]:
# above is same as solution.xc i think.

# xc = [sample(solution.xc, t) for t in solution.td]
# xmatrix = transpose(reduce(hcat, xc));

In [ ]:
println(norm(xmatrix - solution.xd', Inf))
println(norm(xmatrix - solution.xd'))

v_N = [0;0;0]
println("Cost: $(norm(xmatrix[end, 4:6] - v_N))")

In [ ]:
# solution.xd[17:19, 1] + solution.ud[1:3, 1] * dt + dt^2 /2 * (solution.ud[1:3, 2] - solution.ud[1:3, 1]) / dt
# solution.xd[14:16, 1] + solution.xd[17:19, 1] * dt + solution.ud[1:3, 1] * dt^2/2 + dt^3 / 6 * (solution.ud[1:3, 2] - solution.ud[1:3, 1]) / dt

# solution.ud[1:3, 1] * dt^2/2

In [ ]:
# xmatrix - solution.xd'
show(stdout, "text/plain", xmatrix - solution.xd')

In [ ]:
count(abs.(norm.(eachcol(solution.xd[7:10, :]))) .- 1 .> 1e-6) # check if quat norm is within 1e-6 of 1.